<a href="https://colab.research.google.com/github/tushar8049/Sentiment-Classification/blob/master/Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sklearn
!pip install torchvision
!pip install pytorch-nlp
!git clone https://github.com/tushar8049/Sentiment-Classification.git

     |████████████████████████████████| 92kB 3.0MB/s 
Cloning into 'Sentiment-Classification'...
remote: Enumerating objects: 2419, done.
remote: Counting objects: 100% (2419/2419), done.
remote: Compressing objects: 100% (2380/2380), done.
remote: Total 2419 (delta 31), reused 2416 (delta 28), pack-reused 0
Receiving objects: 100% (2419/2419), 5.70 MiB | 13.30 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [0]:
import numpy as np
import pandas as pd
import tarfile
import sys
import os
import io
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import re
from sklearn import preprocessing
from torchnlp import encoders
from torchnlp.encoders import text


Loading the Data in an Array Temporarily after iterating through the files. Along with that encoding of the String is done giving a unique integer to the words in the Corpus. 

*   **raw_review:** determines the bag of sentences both negative and positive classification

*   **encoded_review:** determines the encoded bag of sentences both negative and positive classification

*   **raw_label:** determines the label {negative, positive}

*   **encoded_label:** determines the label {0, 1}

**Location of Data:** Data is primararily hosted on the Github and then the Github repo is cloned and the data is transfered to the the Local Colab Environment.

In [0]:
raw_review = []
raw_label = []

encoded_review = []
encoded_label = []

vocab = {}
vocab_counter = 1
maximum_file_length = 0
# Negative Dataset
path = "/content/Sentiment-Classification/movie_reviews/neg"
for filename in os.listdir(path):
    filename = os.path.join(path, filename)
    reader = io.open(filename)
    line = reader.readline()
    bagOfWords = []
    bagOfEncode = []
    while line:
        sentence = re.sub('[^A-Za-z0-9]+', ' ', line)
        sentence = re.sub(' +', ' ',sentence)
        words = sentence.strip().lstrip().rstrip().split(" ")
        for word in words:
            if word not in vocab.keys():
                vocab[word] = vocab_counter
                vocab_counter += 1
            bagOfEncode.append(vocab[word])
            bagOfWords.append(word)
        line = reader.readline()
    maximum_file_length = max(maximum_file_length, len(bagOfEncode))
    raw_review.append(bagOfWords)
    encoded_review.append(bagOfEncode)
    raw_label.append("NEGATIVE")
    encoded_label.append(0)

# Positive Dataset
path = "/content/Sentiment-Classification/movie_reviews/pos"
for filename in os.listdir(path):
    filename = os.path.join(path, filename)
    reader = io.open(filename)
    line = reader.readline()
    bagOfWords = []
    bagOfEncode = []
    while line:
        sentence = re.sub('[^A-Za-z0-9]+', ' ', line)
        sentence = re.sub(' +', ' ',sentence)
        words = sentence.strip().lstrip().rstrip().split(" ")
        for word in words:
            if word not in vocab.keys():
                vocab[word] = vocab_counter
                vocab_counter += 1
            bagOfEncode.append(vocab[word])
            bagOfWords.append(word)
        line = reader.readline()
    maximum_file_length = max(maximum_file_length, len(bagOfEncode))
    raw_review.append(bagOfWords)
    encoded_review.append(bagOfEncode)
    raw_label.append("POSITIVE")
    encoded_label.append(1)

print("Length of Data: ", len(raw_review))
print("Length of Labels: ", len(raw_label))

print("Length of Data: ", len(encoded_review))
print("Length of Labels: ", len(encoded_label))

print("Vocab Length: ",len(vocab))
print("File with maximum length: ", maximum_file_length)

Length of Data:  2000
Length of Labels:  2000
Length of Data:  2000
Length of Labels:  2000
Vocab Length:  39400
File with maximum length:  2462


Data Loading is done for the Test Data and seperate Arrays have been created for that.
*   **test_raw_review:** determines the bag of sentences both negative and positive classification

*   **test_encoded_review:** determines the encoded bag of sentences both negative and positive classification

*   **test_raw_label:** determines the label {negative, positive}

*   **test_encoded_label:** determines the label {0, 1}

In [0]:
test_raw_review = []
test_raw_label = []

test_encoded_review = []
test_encoded_label = []


# Negative Dataset
path = "/content/Sentiment-Classification/movie_reviews/test/neg"
for filename in os.listdir(path):
    filename = os.path.join(path, filename)
    reader = io.open(filename)
    line = reader.readline()
    bagOfWords = []
    bagOfEncode = []
    while line:
        sentence = re.sub('[^A-Za-z0-9]+', ' ', line)
        sentence = re.sub(' +', ' ',sentence)
        words = sentence.strip().lstrip().rstrip().split(" ")
        for word in words:
            if word not in vocab.keys():
                bagOfEncode.append(vocab[word])
            else:
                bagOfEncode.append(0)
            bagOfWords.append(word)
        line = reader.readline()
    test_raw_review.append(bagOfWords)
    test_encoded_review.append(bagOfEncode)
    test_raw_label.append("NEGATIVE")
    test_encoded_label.append(0)

# Positive Dataset
path = "/content/Sentiment-Classification/movie_reviews/test/pos"
for filename in os.listdir(path):
    filename = os.path.join(path, filename)
    reader = io.open(filename)
    line = reader.readline()
    bagOfWords = []
    bagOfEncode = []
    while line:
        sentence = re.sub('[^A-Za-z0-9]+', ' ', line)
        sentence = re.sub(' +', ' ',sentence)
        words = sentence.strip().lstrip().rstrip().split(" ")
        for word in words:
            if word not in vocab.keys():
                bagOfEncode.append(vocab[word])
            else:
                bagOfEncode.append(0)
            bagOfWords.append(word)
        line = reader.readline()
    test_raw_review.append(bagOfWords)
    test_encoded_review.append(bagOfEncode)
    test_raw_label.append("POSITIVE")
    test_encoded_label.append(1)

print("Length of Test Data: ", len(test_raw_review))
print("Length of Test Labels: ", len(test_raw_label))

print("Length of Test Encoded Data: ", len(test_encoded_review))
print("Length of Test Encoded Labels: ", len(test_encoded_label))

print("File with maximum length: ", maximum_file_length)

Length of Test Data:  100
Length of Test Labels:  100
Length of Test Encoded Data:  100
Length of Test Encoded Labels:  100
File with maximum length:  2462


Padding **encoded_review** so that the dimension of the Array of Arrays become equal as not all files have equal number of words and Neural Net requires input of same length.

In [0]:
encoded_review = np.asarray([np.pad(a, (0, maximum_file_length - len(a)), 'constant', constant_values=0) for a in encoded_review])
encoded_review

encoded_label = np.asarray([encoded_label])


test_encoded_review = np.asarray([np.pad(a, (0, maximum_file_length - len(a)), 'constant', constant_values=0) for a in test_encoded_review])
test_encoded_review

test_encoded_label = np.asarray([test_encoded_label])


In [0]:
encoded_review

array([[   1,    2,    3, ...,    0,    0,    0],
       [ 292,  170,  197, ...,    0,    0,    0],
       [ 156,  447,   11, ...,    0,    0,    0],
       ...,
       [6957,   49,   23, ...,    0,    0,    0],
       [   1,    2,  165, ...,    0,    0,    0],
       [ 140,  257,  471, ...,    0,    0,    0]])

Spliting the Dataset into Training and Validating Data ans also Shuffling both the test data and training and validating data.

In [0]:
print("Review: ", encoded_review.shape)

combined_encoded_data = np.concatenate((encoded_review, encoded_label.T), axis=1)
combined_encoded_data

print("Combined Data for Shuffling: ", combined_encoded_data.shape)

np.random.shuffle(combined_encoded_data)


train_ratio = 0.8
valid_ratio = 0.2

train_cutoff = int(len(combined_encoded_data) * train_ratio)

training_data = combined_encoded_data[:train_cutoff]
validating_data = combined_encoded_data[train_cutoff:]

training_review = training_data[:,0:-1]
training_label = training_data[:,-1]

validating_review = validating_data[:,0:-1]
validating_label = validating_data[:,-1]

print("Training Data Shape: ", training_review.shape)
print("Training Label Shape: ", training_label.shape)

print("Validating Data Shape: ", validating_review.shape)
print("Validating Label Shape: ", validating_label.shape)


combined_test_encoded_data = np.concatenate((test_encoded_review, test_encoded_label.T), axis=1)
combined_test_encoded_data

np.random.shuffle(combined_test_encoded_data)
test_review = combined_test_encoded_data[:,0:-1]
test_label = combined_test_encoded_data[:,-1]
print("Testing Data Shape: ", test_review.shape)
print("Testing Label Shape: ", test_label.shape)

Review:  (2000, 2462)
Combined Data for Shuffling:  (2000, 2463)
Training Data Shape:  (1600, 2462)
Training Label Shape:  (1600,)
Validating Data Shape:  (400, 2462)
Validating Label Shape:  (400,)
Testing Data Shape:  (100, 2462)
Testing Label Shape:  (100,)


Creating an Embedding Dictionary and also transforming the encoded_review into a tensorEncoded format.

In [0]:
embedding = nn.Embedding(len(vocab)+1, 300)
input = torch.LongTensor(encoded_review)

embedded_vocab = {}
embedded_vocab_array =[]
for key in vocab.keys():
    embedded_vocab[key] = embedding(torch.LongTensor([vocab[key]]))
    embedded_vocab_array.append(embedded_vocab[key])
    
    
tensorEncoded = embedding(input)
tensorEncoded

embedding_label = nn.Embedding(2, 300)
input_label = torch.LongTensor(encoded_label)
tensorEncoded_label = embedding_label(input_label)
tensorEncoded_label

tensor([[[-0.2124, -0.8379,  0.6749,  ...,  0.6871, -0.5017, -1.6020],
         [-0.2124, -0.8379,  0.6749,  ...,  0.6871, -0.5017, -1.6020],
         [-0.2124, -0.8379,  0.6749,  ...,  0.6871, -0.5017, -1.6020],
         ...,
         [ 0.0449, -2.2014,  1.9208,  ...,  0.6559,  0.4940,  0.9216],
         [ 0.0449, -2.2014,  1.9208,  ...,  0.6559,  0.4940,  0.9216],
         [ 0.0449, -2.2014,  1.9208,  ...,  0.6559,  0.4940,  0.9216]]],
       grad_fn=<EmbeddingBackward>)

In [0]:
train_data = TensorDataset(torch.from_numpy(training_review), torch.from_numpy(training_label))
valid_data = TensorDataset(torch.from_numpy(validating_review), torch.from_numpy(validating_label))
test_data = TensorDataset(torch.from_numpy(test_review), torch.from_numpy(test_label))

batch_size = 100
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = True)

In [0]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


# **Baseline Model:**

In [0]:
# Defineing Model
model = nn.Sequential(nn.Linear(2462, 512),
                      nn.ReLU(),
                      nn.Linear(512, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 128),
                      nn.ReLU(),
                      nn.Linear(128, 256),
                      nn.ReLU(),
                      nn.Linear(256, 512),
                      nn.ReLU(),
                      nn.Linear(512, 64),
                      nn.ReLU(),
                      nn.Linear(64, 2),
                      nn.Softmax())
print("\nModel Structure:")
print(model)


# Defining Loss function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)



# Training the Model with the number of Epochs
print("\nModel Training:")
counter = 0
epochs = 50
print_every = 4
for e in range(epochs):
    running_loss = 0
    for data, labels in train_loader:
        counter += 1
        optimizer.zero_grad()
        data = data.float()
        labels = labels.float()
        output = model.forward(data)
        loss = criterion(output.float(), labels.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if counter % print_every == 0:
            
            num_correct = 0
            
            val_losses = []
            for inputs, labels in valid_loader:
                
                inputs, labels = inputs.float(), labels.float()

                output = model.forward(data)
                
                pred = torch.round(output.squeeze()) 
                
                actual =[]
                for x,y in pred:
                    if x == 1:
                        actual.append(0)
                    else:
                        actual.append(1)
                actual = torch.FloatTensor(actual)
                # compare predictions to true label
                correct_tensor = actual.eq(labels.float())
                correct = np.squeeze(correct_tensor.numpy())
                num_correct += np.sum(correct)
                val_loss = criterion(output.float(), labels.long())

                val_losses.append(val_loss.item())
                
            
            val_acc = num_correct/len(valid_loader.dataset)
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Acc: {:.6f}".format(np.mean(val_acc)))
            
            
print("\nTest Data Result:")
for inputs, labels in test_loader:
                
    inputs, labels = inputs.float(), labels.float()

    output = model.forward(data)
    
    pred = torch.round(output.squeeze()) 
    
    actual =[]
    for x,y in pred:
        if x == 1:
            actual.append(0)
        else:
            actual.append(1)
    actual = torch.FloatTensor(actual)
    
    correct_tensor = actual.eq(labels.float())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)
    val_loss = criterion(output.float(), labels.long())

    val_losses.append(val_loss.item())


val_acc = num_correct/len(valid_loader.dataset)
print("Test Loss: {:.6f}".format(np.mean(val_losses)),
      "Test Acc: {:.6f}".format(np.mean(val_acc)))


Model Structure:
Sequential(
  (0): Linear(in_features=2462, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=64, bias=True)
  (7): ReLU()
  (8): Linear(in_features=64, out_features=128, bias=True)
  (9): ReLU()
  (10): Linear(in_features=128, out_features=256, bias=True)
  (11): ReLU()
  (12): Linear(in_features=256, out_features=512, bias=True)
  (13): ReLU()
  (14): Linear(in_features=512, out_features=64, bias=True)
  (15): ReLU()
  (16): Linear(in_features=64, out_features=2, bias=True)
  (17): Softmax(dim=None)
)

Model Training:
Epoch: 1/50... Step: 4... Loss: 0.728305... Val Loss: 0.715395 Val Acc: 0.520000


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 1/50... Step: 8... Loss: 0.699181... Val Loss: 0.709362 Val Acc: 0.510000
Epoch: 1/50... Step: 12... Loss: 0.716945... Val Loss: 0.705342 Val Acc: 0.510000
Epoch: 1/50... Step: 16... Loss: 0.671973... Val Loss: 0.721591 Val Acc: 0.510000
Epoch: 2/50... Step: 20... Loss: 0.729916... Val Loss: 0.730440 Val Acc: 0.495000
Epoch: 2/50... Step: 24... Loss: 0.634154... Val Loss: 0.720465 Val Acc: 0.515000
Epoch: 2/50... Step: 28... Loss: 0.665011... Val Loss: 0.719696 Val Acc: 0.480000
Epoch: 2/50... Step: 32... Loss: 0.663413... Val Loss: 0.714078 Val Acc: 0.505000
Epoch: 3/50... Step: 36... Loss: 0.633806... Val Loss: 0.708405 Val Acc: 0.530000
Epoch: 3/50... Step: 40... Loss: 0.650462... Val Loss: 0.692378 Val Acc: 0.550000
Epoch: 3/50... Step: 44... Loss: 0.629027... Val Loss: 0.727637 Val Acc: 0.480000
Epoch: 3/50... Step: 48... Loss: 0.617890... Val Loss: 0.736043 Val Acc: 0.500000
Epoch: 4/50... Step: 52... Loss: 0.611092... Val Loss: 0.717330 Val Acc: 0.495000
Epoch: 4/50... St

# **RNN with BaseLine Model:**

In [0]:
#Defining the Model
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        batch_size = x.size(0)

        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sig(out)
        
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
    
    
# Instantiating the model with hyperparams
vocab_size = len(vocab)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 128
n_layers = 5

print("\nModel Structure:")
net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)


# Defining the Loss Functions and Optimizer
lr=0.001
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=lr)


# Training the Model with the number of Epochs
print("\nModel Training:")
epochs = 4 
counter = 0
print_every = 4
clip=5 

if(train_on_gpu):
    net.cuda()

net.train()
for e in range(epochs):
    h = net.init_hidden(batch_size)

    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        h = tuple([each.data for each in h])
        net.zero_grad()
        output, h = net(inputs, h)

        loss = criterion(output.squeeze(), labels.float())
        loss.backward()

        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        if counter % print_every == 0:
            num_correct = 0
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                pred = torch.round(output.squeeze())

                correct_tensor = pred.eq(labels.float().view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
                num_correct += np.sum(correct)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())
                
            val_acc = num_correct/len(valid_loader.dataset)
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Acc: {:.6f}".format(np.mean(val_acc)))
            

print("\nTest Data Result:")
for inputs, labels in test_loader:
    val_h = tuple([each.data for each in val_h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    output, val_h = net(inputs, val_h)
    pred = torch.round(output.squeeze())

    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
    val_loss = criterion(output.squeeze(), labels.float())

    val_losses.append(val_loss.item())

val_acc = num_correct/len(valid_loader.dataset)
net.train()
print("Test Loss: {:.6f}".format(np.mean(val_losses)),
      "Test Acc: {:.6f}".format(np.mean(val_acc)))


Model Structure:
SentimentRNN(
  (embedding): Embedding(39401, 400)
  (lstm): LSTM(400, 128, num_layers=5, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)

Model Training:
Epoch: 1/4... Step: 4... Loss: 0.700895... Val Loss: 0.693502 Val Acc: 0.495000
Epoch: 1/4... Step: 8... Loss: 0.691015... Val Loss: 0.693109 Val Acc: 0.505000
Epoch: 1/4... Step: 12... Loss: 0.695237... Val Loss: 0.693110 Val Acc: 0.505000
Epoch: 1/4... Step: 16... Loss: 0.694471... Val Loss: 0.693220 Val Acc: 0.495000
Epoch: 2/4... Step: 20... Loss: 0.695540... Val Loss: 0.693118 Val Acc: 0.505000
Epoch: 2/4... Step: 24... Loss: 0.691718... Val Loss: 0.693203 Val Acc: 0.495000
Epoch: 2/4... Step: 28... Loss: 0.691123... Val Loss: 0.693209 Val Acc: 0.495000
Epoch: 2/4... Step: 32... Loss: 0.694246... Val Loss: 0.693291 Val Acc: 0.495000
Epoch: 3/4... Step: 36... Loss: 0.694573... Val Loss: 0.693548 Val Acc: 0.4

# **RNN with BaseLine Model:**

In [0]:
embed_size = 300 
max_features = len(vocab) + 1
maxlen = 70 
batch_size = 100 
n_epochs = 5
n_splits = 5 
SEED = 10
debug = 0
embedding_matrix = encoded_review

class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

class Attention_Net(nn.Module):
    def __init__(self):
        super(Attention_Net, self).__init__()
        drp = 0.1
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False

        self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(embed_size, 128, bidirectional=True, batch_first=True)
        self.lstm2 = nn.GRU(128*2, 64, bidirectional=True, batch_first=True)

        self.attention_layer = Attention(128, maxlen)
        
        self.linear = nn.Linear(64*2 , 64)
        self.relu = nn.ReLU()
        self.out = nn.Linear(64, 1)

    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        h_lstm, _ = self.lstm2(h_lstm)
        h_lstm_atten = self.attention_layer(h_lstm)
        conc = self.relu(self.linear(h_lstm_atten))
        out = self.out(conc)
        return out
    
model = Attention_Net()
print(model)

# Defining Loss function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)



# Training the Model with the number of Epochs
print("\nModel Training:")
counter = 0
epochs = 5
print_every = 4
for e in range(epochs):
    running_loss = 0
    for data, labels in train_loader:
        counter += 1
        optimizer.zero_grad()
        data = data.float()
        labels = labels.float()
        output = model.forward(data)
        loss = criterion(output.float(), labels.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if counter % print_every == 0:
            
            num_correct = 0
            
            val_losses = []
            for inputs, labels in valid_loader:
                
                inputs, labels = inputs.float(), labels.float()

                output = model.forward(data)
                
                pred = torch.round(output.squeeze()) 
                
                actual =[]
                for x,y in pred:
                    if x == 1:
                        actual.append(0)
                    else:
                        actual.append(1)
                actual = torch.FloatTensor(actual)
                # compare predictions to true label
                correct_tensor = actual.eq(labels.float())
                correct = np.squeeze(correct_tensor.numpy())
                num_correct += np.sum(correct)
                val_loss = criterion(output.float(), labels.long())

                val_losses.append(val_loss.item())
                
            
            val_acc = num_correct/len(valid_loader.dataset)
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Acc: {:.6f}".format(np.mean(val_acc)))
            
            
print("\nTest Data Result:")
for inputs, labels in test_loader:
                
    inputs, labels = inputs.float(), labels.float()

    output = model.forward(data)
    
    pred = torch.round(output.squeeze()) 
    
    actual =[]
    for x,y in pred:
        if x == 1:
            actual.append(0)
        else:
            actual.append(1)
    actual = torch.FloatTensor(actual)
    
    correct_tensor = actual.eq(labels.float())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)
    val_loss = criterion(output.float(), labels.long())

    val_losses.append(val_loss.item())


val_acc = num_correct/len(valid_loader.dataset)
print("Test Loss: {:.6f}".format(np.mean(val_losses)),
      "Test Acc: {:.6f}".format(np.mean(val_acc)))

Attention_Net(
  (embedding): Embedding(39401, 300)
  (embedding_dropout): Dropout2d(p=0.1, inplace=False)
  (lstm): LSTM(300, 128, batch_first=True, bidirectional=True)
  (lstm2): GRU(256, 64, batch_first=True, bidirectional=True)
  (attention_layer): Attention()
  (linear): Linear(in_features=128, out_features=64, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=64, out_features=1, bias=True)
)

Model Training:


RuntimeError: ignored